<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#The-Catalog-Service-API" data-toc-modified-id="The-Catalog-Service-API-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>The Catalog Service API</a></span></li></ul></div>

In [1]:
import aepp
from aepp import catalog

# The Catalog Service API

The Catalog API is a service that would help investigating the data lake catalog that is available.\
In the Catalog API, you can access the datasets or the batches that are ingested in your organization.\
Catalog is the system of record for data location and lineage within Adobe Experience Platform.\ Catalog Service does not contain the actual files or directories that contain the data. Instead, it holds the metadata and description of those files and directories.

Catalog acts as a metadata store or "catalog" where you can find information about your data within Experience Platform.

Use Catalog to answer the following questions: 
* Where is my data located? 
* At what stage of processing is this data? 
* What systems or processes have acted on my data? 
* What errors occurred during processing? 
* If successful, how much data was processed?